In [1]:
import json
import pickle

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from indra_db.util.content_scripts import get_text_content_from_pmids
from indra_db.util.content_scripts import get_stmts_with_agent_text_like
from indra_db.util.content_scripts import get_text_content_from_stmt_ids

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator

from adeft_indra.s3 import model_to_s3
from adeft_indra.ground import gilda_ground

INFO: [2020-01-16 15:56:38] /Users/albertsteppi/adeft/adeft/recognize.py - OneShotRecognizer not available. AdeftLongformScorer has not been built successfully.


In [2]:
shortforms = ['TEK']
genes = ['TEK']
families = {}
groundings = [f'HGNC:{get_hgnc_id(gene)}' for gene in genes]
for family, members in families.items():
    genes.extend(members)
    groundings.append(f'FPLX:{family}')
with open('../data/entrez_all_pmids.json', 'r') as f:
    all_pmids = json.load(f)

In [3]:
genes

['TEK']

In [4]:
entrez_texts = []
entrez_refs = set()
for gene, grounding in zip(genes, groundings):
    try:
        pmids = all_pmids[gene]
    except KeyError:
        continue
    _, content = get_text_content_from_pmids(pmids)
    entrez_texts.extend([(universal_extract_text(text), grounding)
                          for text in content.values() if text])
    entrez_refs.update(content.keys())

In [5]:
miners = dict()
all_texts = set()
for shortform in shortforms:
    stmts = get_stmts_with_agent_text_like(shortform)[shortform]
    _, content = get_text_content_from_stmt_ids(stmts)
    shortform_texts = [universal_extract_text(text, contains=shortforms)
                       for ref, text in content.items() if text and ref not in entrez_refs]
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(shortform_texts)
    all_texts.union(shortform_texts)

It's then necessary to check if Acromine produced the correct results. We must fix errors manually

In [10]:
top = miners['TEK'].top()

In [11]:
top

[('transposable element silencing via at hook', 6.0),
 ('traditional ecological knowledge', 4.0),
 ('of traditional ecological knowledge', 2.0),
 ('protein transposable element silencing via at hook', 1.3333333333333333),
 ('receptor', 1.0),
 ('tek tyrosine kinase endothelial', 1.0),
 ('we test whether traditional ecological knowledge', 1),
 ('of transposable element silencing via at hook', 1.0),
 ('binding protein transposable element silencing via at hook', 1.0),
 ('een biological and cultural realms and these experiences construe traditional ecological knowledge',
  1),
 ('nism of reciprocity unsatisfactory for the contemporary holders of traditional ecological knowledge',
  1),
 ('plexes in relation with angiogenesis and vascular maturation processes the angiopoietin 1 receptor',
  1),
 ('dh angiopoietin 1 angpt1 tgf β1 tgfb1 contactin 1 cntn1 tek tyrosine kinase endothelial',
  1),
 ('g42940 see below which was previously designated as transposable element silencing via at hook',


In [12]:
longforms0 = miners['TEK'].get_longforms()

In [13]:
longforms0

[('transposable element silencing via at hook', 6.0),
 ('traditional ecological knowledge', 4.0)]

In [35]:
longforms0 = [x for i, x in enumerate(longforms0) if i in [0, 2, 3, 4]]

In [36]:
longforms0.extend([(longform, score) for i, (longform, score) in enumerate(top)
                   if i in [2, 3]])

In [37]:
longforms0.sort(key=lambda x: -x[1])

In [14]:
longforms, scores = zip(*longforms0)

In [15]:
longforms

('transposable element silencing via at hook',
 'traditional ecological knowledge')

In [16]:
grounding_map = {}
names = {}
for longform in longforms:
    grounding = gilda_ground(longform)
    if grounding[0]:
        grounding_map[longform] = f'{grounding[0]}:{grounding[1]}'
        names[grounding_map[longform]] = grounding[2]

In [17]:
grounding_map

{}

In [18]:
names

{}

In [19]:
grounding_map, names, pos_labels = ground_with_gui(longforms, scores, grounding_map=grounding_map, names=names)

In [20]:
names

{}

In [21]:
grounding_map

{'traditional ecological knowledge': 'ungrounded',
 'transposable element silencing via at hook': 'ungrounded'}

In [22]:
names['HGNC:11724'] = 'TEK'
pos_labels.append('HGNC:11724')

In [23]:
result = (grounding_map, names, pos_labels)

In [24]:
result

({'traditional ecological knowledge': 'ungrounded',
  'transposable element silencing via at hook': 'ungrounded'},
 {'HGNC:11724': 'TEK'},
 ['HGNC:11724'])

In [25]:
grounding_map, names, pos_labels = ({'traditional ecological knowledge': 'ungrounded',
  'transposable element silencing via at hook': 'ungrounded'},
 {'HGNC:11724': 'TEK'},
 ['HGNC:11724'])

In [26]:
grounding_dict = {'TEK': grounding_map}

In [27]:
classifier = AdeftClassifier('TEK', pos_labels=pos_labels)

In [28]:
param_grid = {'C': [100.0], 'max_features': [10000]}

In [29]:
labeler = AdeftLabeler(grounding_dict)

In [30]:
corpus = labeler.build_from_texts(shortform_texts)

In [31]:
corpus.extend(entrez_texts)

In [32]:
texts, labels = zip(*corpus)

In [33]:
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=8)

INFO: [2020-01-16 15:59:18] /Users/albertsteppi/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-01-16 15:59:21] /Users/albertsteppi/adeft/adeft/modeling/classify.py - Best f1 score of 0.9931551078393387 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [34]:
classifier.stats

{'label_distribution': {'ungrounded': 9, 'HGNC:11724': 217},
 'f1': {'mean': 0.9931551078393387, 'std': 0.005589626835700498},
 'precision': {'mean': 0.9864646464646466, 'std': 0.011053108477694943},
 'recall': {'mean': 1.0, 'std': 0.0}}

In [35]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [36]:
d = disamb.disambiguate(shortform_texts)

In [37]:
a = [text for pred, text in zip(d, shortform_texts)if pred[0] == 'ungrounded']

In [38]:
len(a)

12

In [88]:
a[16]

'The OsPCS2 exhibits root- and shoot-specific differential ratios of alternatively spliced transcripts in indica rice under Cd stress, and plays role in Cd and As stress tolerance and accumulation. Enzymatic activity of phytochelatin synthase (PCS) in plant produces phytochelatins, which help in sequestration of heavy metal(loid)s inside the cell vacuole to alleviate toxicity. Here we report that among the two PCS genes-OsPCS1 and OsPCS2 in indica rice (Oryza sativa) cultivar, the OsPCS2 produces an alternatively spliced OsPCS2b transcript that bears the unusual premature termination codon besides the canonically spliced OsPCS2a transcript. Root- and shoot-specific differential ratios of alternatively spliced OsPCS2a and OsPCS2b transcript expressions were observed under cadmium stress. Saccharomyces cerevisiae cells transformed with OsPCS2a exhibited increased cadmium (Cd) and arsenic (As) tolerance and accumulation, unlike the OsPCS2b transformed yeast cells. An intron-containing hai

In [86]:
disamb.dump('CKI', '../results')

In [ ]:
from adeft.disambiguate import load_disambiguator

In [ ]:
d = load_disambiguator('ARG', '../results')

In [ ]:
d.disambiguate(texts[0])

In [ ]:
print(d.info())

In [ ]:
a = load_disambiguator('AR')

In [ ]:
a.disambiguate('Androgen')

In [ ]:
logit = d.classifier.estimator.named_steps['logit']

In [ ]:
logit.classes_

In [87]:
model_to_s3(disamb)